# DATA PREPARATION AND METHODS LSTM

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Wczytanie danych z przegrzewacza
data = pd.read_csv("..\dane\przegrzewacz\oper_6-ACT-1-B_1-2.txt", sep='\t').reset_index()
columns = list(data.columns)
#There is no index column so we have to create one
# Shift the labels
columns = columns[1:] + [columns[0]]
# Assign the new labels back to the DataFrame
data.columns = columns
# Drop the last column
data = data.drop(data.columns[-1],axis=1)

In [ ]:
data.head(10)

In [ ]:
plt.plot(data['TS31P'])

In [ ]:
#Słownik z zakresami dla danych z każdego kolumuny (zakresy podane przez przemysł)
ranges = {
  't':(0, 0), 'B':(0, 100), 'FP4P':(30, 140), 'TS31P':(395, 445), 'GV3':(0, 100), 'TS32P':(385, 435), 'TP32P':(465, 515), 'GV4':(0, 100), 'TS42P':(455, 505),
       'TP42P':(515, 565), 'SP31':(0, 0), 'CV31':(0, 100), 'SP32':(0, 0), 'CV32':(0, 100), 'SP41':(0, 0), 'CV41':(0, 100), 'SP42':(0, 0), 'CV42':(0, 100),
       'FP3P':(30, 140), 'FP2P':(30, 140)
}

In [ ]:
#Normalizacja danych na podstawie podanych zakresów->znormalizowane dane zostają zapisane do data_norm Dataframe
data_norm = data.copy()
for column in data_norm:
  if(ranges[column] != (0, 0)): #jeśli jest podany zakres
    data_norm[[column]] = (data_norm[[column]]-ranges[column][0])/(ranges[column][1]-ranges[column][0])
  
data_norm.head()

In [ ]:
#Resampling the data -> taking every 10th row
data_resampled = data_norm.iloc[::10]
data_resampled.reset_index(inplace= True, drop= True)

data_resampled_raw = data.iloc[::10]
data_resampled_raw.reset_index(inplace= True, drop= True)

In [ ]:
#Funkcja odpowiedzialna za denormalizację wyników
def denorm(data_norm, pred_name):
  data_denorm = data_norm.copy()
  for column in data_denorm:
    if(ranges[pred_name] != (0, 0)): #jeśli jest podany zakres
      data_denorm[[column]] = (data_denorm[[column]]*(ranges[pred_name][1]-ranges[pred_name][0]))+ranges[pred_name][0]
  return data_denorm

In [ ]:
#Wyodrębnienie zbiorów danych niezbędnych do stworzenia każdego z modeli
#Model temperatury za schładzaczem 3
df_TS32P_raw = data[['TS32P', 'TS31P', 'GV3']]
df_TS32P = data_norm[['TS32P', 'TS31P', 'GV3']]
df_TS32P_resampled = data_resampled[['TS32P', 'TS31P', 'GV3']]
df_TS32P_resampled_raw = data_resampled_raw[['TS32P', 'TS31P', 'GV3']]

#Model temperatury za przegrzewaczem 3
df_TP32P_raw = data[['TP32P', 'TS32P', 'B', 'FP3P']]
df_TP32P = data_norm[['TP32P', 'TS32P', 'B', 'FP3P']]
df_TP32P_resampled = data_resampled[['TP32P', 'TS32P', 'B', 'FP3P']]
df_TP32P_resampled_raw = data_resampled_raw[['TP32P', 'TS32P', 'B', 'FP3P']]

#Model temperatury za schładzaczem 4
df_TS42P_raw = data[['TS42P', 'GV4', 'FP3P','TP32P']]
df_TS42P = data_norm[['TS42P', 'GV4', 'FP3P','TP32P']] #, 'TS41P'='TP32P'
df_TS42P_resampled = data_resampled[['TS42P', 'GV4', 'FP3P','TP32P']]
df_TS42P_resampled_raw = data_resampled_raw[['TS42P', 'GV4', 'FP3P','TP32P']]

#Model temperatury za przegrzewaczem 4
df_TP42P_raw = data[['TP42P', 'TS42P', 'B', 'FP4P']]
df_TP42P = data_norm[['TP42P', 'TS42P', 'B', 'FP4P']]
df_TP42P_resampled = data_resampled[['TP42P', 'TS42P', 'B', 'FP4P']]
df_TP42P_resampled_raw = data_resampled_raw[['TP42P', 'TS42P', 'B', 'FP4P']]

In [ ]:
#Funkcja przetwarzająca szereg czasowy na próbki w postaci: window_size wartości poprzednich -> wartość następna
def data_to_X_y(data, window_size=5):
    data_as_np = data.to_numpy()
    X = []
    y = []
    for i in range(len(data_as_np)-window_size):
        #Jako wejście bierzemy ciąg o długości WINDOW_SIZE-1
        row = [[a] for a in data_as_np[i:i+window_size]]
        X.append(row)
        #Jako etykietę bierzemy następną wartość po ciągu
        label = data_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)


In [ ]:
#Funkcja odpowiedzialna za podział danych na zbiór treningowy, walidacyjny oraz testowy (przetwarzane są szeregi czasowe, więc nie mieszamy danych)
def data_to_cross_val(sequences, variables, labels):
  #Pobieramy rozmiar zbioru danych wejściowych
  size = sequences.shape[0]
  train = int(size*0.8)
  val = int(size*0.9)
  
  sequences_train, variables_train, labels_train = sequences[:train], variables[:train], labels[:train]
  sequences_val, variables_val, labels_val = sequences[train:val], variables[train:val], labels[train:val]
  sequences_test, variables_test, labels_test = sequences[val:], variables[val:], labels[val:]
  
  return sequences_train, variables_train, labels_train, sequences_val, variables_val, labels_val, sequences_test, variables_test, labels_test
  

In [ ]:
#Keras imports
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.models import load_model

In [ ]:
#Funkcja odpowiedzialna za tworzenie modelu
def create_model(var_num):
    # Define input layers
    sequence_input = Input(shape=(None, 1))  #Values for the past time steps (input can have any window size)
    variables_input = Input(shape=(var_num,))  # Variables for the current time step

    # LSTM layers with dropout for sequences
    lstm_sequence = LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(sequence_input)
    lstm_sequence = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(lstm_sequence)

    #Dense layer for additional variables at current time step
    dense_variables = Dense(32, activation='relu')(variables_input)
    dense_variables = Dropout(0.2)(dense_variables)
    #scaled_variables = TrainableScaleLayer()(dense_variables)
    
    # Concatenate the outputs of both layers
    merged = concatenate([lstm_sequence, dense_variables])
    
    # Dense layer for regression with linear activation function
    output = Dense(1, activation='linear')(merged)

    # Create the model
    model = Model(inputs=[sequence_input, variables_input], outputs=output)
    return model

In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości -> wersja ze średnią ważoną
def predict(model, input_sequence, input_variables, window_size, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    sequence_input = np.array(input_sequence[0]) #We take only the first window_size values from the data
    variables_input = np.array(input_variables) #We create a temporary variable for additional variables data
    
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict([sequence_input.reshape(1, window_size, 1), variables_input[i].reshape(1,variables_input.shape[1])], verbose=0) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0][0]
        
        # Update the input sequence for the next iteration
        sequence_input = np.roll(sequence_input, -1)  # Shift left one place
        sequence_input[-1] = ((9 * next_value[0][0] + labels[i]) / 10)  # Insert the new value (weighted average) at the end
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions


In [ ]:
# #Funkcja odpowiedzialna za przewidywanie kolejnych wartości -> wersja bez średniej ważonej
# def predict(model, input_sequence, input_variables, window_size, num_steps):
#     predictions = np.zeros(num_steps) #empty numpy array for storing predictions
#     sequence_input = np.array(input_sequence[0]) #We take only the first window_size values from the data
#     variables_input = np.array(input_variables) #We create a temporary variable for additional variables data
    
#     for i in range(num_steps):
        
#         #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
#         next_value = model.predict([sequence_input.reshape(1, window_size, 1), variables_input[i].reshape(1,variables_input.shape[1])], verbose=0) 
        
#         #Adding predicted values to predictions array
#         predictions[i] = next_value[0][0]
        
#         # Update the input sequence for the next iteration
#         sequence_input = np.roll(sequence_input, -1)  # Shift left one place
#         sequence_input[-1] = next_value[0][0]  # Insert the new value at the end
        
#         #Counting how many steps forward have already been predicted and printing it to the console
#         if i % 500 == 0:
#             #os.system('cls')
#             print(i, '/', num_steps)
#     print("Done")
#     return predictions


In [ ]:
#Funkcja odpowedzialna za wyświetlanie wyników
def display(train_results, val_results, test_results, which_dataset, plot_name):
    if which_dataset=='all':
        # Create a figure and subplots for each dataset from cross validation
        fig, axs = plt.subplots(
            3, 2,               # 3 rows, 2 columns
            figsize=(20, 16),    # Width = 10 , height = 8 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0, 0].plot(train_results['Train Predictions'], label='Model')
        axs[0, 0].plot(train_results['Actuals'], label='Real')
        axs[0, 0].set_title('Training dataset')
        axs[0, 0].set_xlabel('Time')
        axs[0, 0].set_ylabel('Temperature')
        axs[0, 0].legend()

        axs[1, 0].plot(val_results['Val Predictions'], label='Model')
        axs[1, 0].plot(val_results['Actuals'], label='Real')
        axs[1, 0].set_title('Validation dataset')
        axs[1, 0].set_xlabel('Time')
        axs[1, 0].set_ylabel('Temperature')
        axs[1, 0].legend()

        axs[2, 0].plot(test_results['Test Predictions'], label='Model')
        axs[2, 0].plot(test_results['Actuals'], label='Real')
        axs[2, 0].set_title('Testing dataset')
        axs[2, 0].set_xlabel('Time')
        axs[2, 0].set_ylabel('Temperature')
        axs[2, 0].legend()

        # Plot difference between model and real data
        axs[0, 1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
        axs[0, 1].set_title('Training dataset (Difference)')
        axs[0, 1].set_xlabel('Time')
        axs[0, 1].set_ylabel('Temperature')
        axs[0, 1].legend()

        axs[1, 1].plot(val_results['Val Predictions']- val_results['Actuals'], label='Difference', color='red' )
        axs[1, 1].set_title('Validation dataset (Difference)')
        axs[1, 1].set_xlabel('Time')
        axs[1, 1].set_ylabel('Temperature')
        axs[1, 1].legend()

        axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
        axs[2, 1].set_title('Testing dataset (Difference)')
        axs[2, 1].set_xlabel('Time')
        axs[2, 1].set_ylabel('Temperature')
        axs[2, 1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
        
    elif which_dataset=='train':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(train_results['Train Predictions'], label='Model')
        axs[0].plot(train_results['Actuals'], label='Real')
        axs[0].set_title('Training dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Training dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    elif which_dataset=='val':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(val_results['Val Predictions'], label='Model')
        axs[0].plot(val_results['Actuals'], label='Real')
        axs[0].set_title('Validation dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(val_results['Val Predictions'] - val_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Validation dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
    
    elif which_dataset=='test':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(test_results['Test Predictions'], label='Model')
        axs[0].plot(test_results['Actuals'], label='Real')
        axs[0].set_title('Testing dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Testing dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    else: print("Please enter a valid dataset to display!")

In [ ]:
#Funkcja odpowiedzialna za wyświetlanie wyników z podanego przedziału
def display_window(train_results, val_results, test_results, which_dataset, plot_name, train=0, val=0, test=0, window_train=3000, window_val=3000, window_test=3000):
    if which_dataset=='all':
        # Create a figure and subplots for each dataset from cross validation
        fig, axs = plt.subplots(
            3, 2,               # 3 rows, 2 columns
            figsize=(20, 16),    # Width = 10 , height = 8 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0, 0].plot(train_results['Train Predictions'][train:train+window_train], label='Model') #
        axs[0, 0].plot(train_results['Actuals'][train:train+window_train], label='Real')
        axs[0, 0].set_title('Training dataset')
        axs[0, 0].set_xlabel('Time')
        axs[0, 0].set_ylabel('Temperature')
        axs[0, 0].legend()

        axs[1, 0].plot(val_results['Val Predictions'][val:val+window_val], label='Model') #
        axs[1, 0].plot(val_results['Actuals'][val:val+window_val], label='Real')
        axs[1, 0].set_title('Validation dataset')
        axs[1, 0].set_xlabel('Time')
        axs[1, 0].set_ylabel('Temperature')
        axs[1, 0].legend()

        axs[2, 0].plot(test_results['Test Predictions'][test:test+window_test], label='Model') #
        axs[2, 0].plot(test_results['Actuals'][test:test+window_test], label='Real')
        axs[2, 0].set_title('Testing dataset')
        axs[2, 0].set_xlabel('Time')
        axs[2, 0].set_ylabel('Temperature')
        axs[2, 0].legend()

        # Plot difference between model and real data
        axs[0, 1].plot(train_results['Train Predictions'][train:train+window_train] - train_results['Actuals'][train:train+window_train], label='Difference', color='red') #
        axs[0, 1].set_title('Training dataset (Difference)')
        axs[0, 1].set_xlabel('Time')
        axs[0, 1].set_ylabel('Temperature')
        axs[0, 1].legend()

        axs[1, 1].plot(val_results['Val Predictions'][val:val+window_val]- val_results['Actuals'][val:val+window_val], label='Difference', color='red' ) #
        axs[1, 1].set_title('Validation dataset (Difference)')
        axs[1, 1].set_xlabel('Time')
        axs[1, 1].set_ylabel('Temperature')
        axs[1, 1].legend()

        axs[2, 1].plot(test_results['Test Predictions'][test:test+window_test] - test_results['Actuals'][test:test+window_test], label='Difference', color='red') #
        axs[2, 1].set_title('Testing dataset (Difference)')
        axs[2, 1].set_xlabel('Time')
        axs[2, 1].set_ylabel('Temperature')
        axs[2, 1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
        
    elif which_dataset=='train':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(train_results['Train Predictions'][train:train+window_train], label='Model')
        axs[0].plot(train_results['Actuals'][train:train+window_train], label='Real')
        axs[0].set_title('Training dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(train_results['Train Predictions'][train:train+window_train] - train_results['Actuals'][train:train+window_train], label='Difference', color='red')
        axs[1].set_title('Training dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    elif which_dataset=='val':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(val_results['Val Predictions'][val:val+window_val], label='Model')
        axs[0].plot(val_results['Actuals'][val:val+window_val], label='Real')
        axs[0].set_title('Validation dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(val_results['Val Predictions'][val:val+window_val] - val_results['Actuals'][val:val+window_val], label='Difference', color='red')
        axs[1].set_title('Validation dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
    
    elif which_dataset=='test':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(test_results['Test Predictions'][test:test+window_test], label='Model')
        axs[0].plot(test_results['Actuals'][test:test+window_test], label='Real')
        axs[0].set_title('Testing dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Temperature')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(test_results['Test Predictions'][test:test+window_test] - test_results['Actuals'][test:test+window_test], label='Difference', color='red')
        axs[1].set_title('Testing dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Temperature')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    else: print("Please enter a valid dataset and parameters to display!")

In [ ]:
#Funkcja odpowiedzialna za zapisywanie dataframe do csv
def save(df, file_name):
  df.to_csv('predictions_new/'+file_name+'.csv')
  

In [ ]:
#Funkcja odpowiedzialna za odczytywanie dataframe z csv
def get(file_name):
  return pd.read_csv('predictions_new/'+file_name+'.csv')

In [ ]:
#Porównanie wykresów trzech różnych modeli na zbiorze testowym + jednostki

def display_comparison(train_results, val_results, test_results, plot_name, y_unit='°C', x_unit='10s'):
    # Create a figure and subplots for each dataset from cross validation
    fig, axs = plt.subplots(
        3, 2,               
        figsize=(20, 16),    
        gridspec_kw=dict(hspace=0.5, wspace=0.5)   
    )

    # Plot data on subplots
    axs[0, 0].plot(train_results['Test Predictions'], label='Model')
    axs[0, 0].plot(train_results['Actuals'], label='Real')
    axs[0, 0].set_title('Testing dataset LSTM')
    axs[0, 0].set_xlabel(f'Time [{x_unit}]')
    axs[0, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[0, 0].legend()

    axs[1, 0].plot(val_results['Test Predictions'], label='Model')
    axs[1, 0].plot(val_results['Actuals'], label='Real')
    axs[1, 0].set_title('Testing dataset MLP ARX')
    axs[1, 0].set_xlabel(f'Time [{x_unit}]')
    axs[1, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[1, 0].legend()

    axs[2, 0].plot(test_results['Test Predictions'], label='Model')
    axs[2, 0].plot(test_results['Actuals'], label='Real')
    axs[2, 0].set_title('Testing dataset LINEAR ARX')
    axs[2, 0].set_xlabel(f'Time [{x_unit}]')
    axs[2, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[2, 0].legend()

    # Plot difference between model and real data
    axs[0, 1].plot(train_results['Test Predictions'] - train_results['Actuals'], label='Difference', color='red')
    axs[0, 1].set_title('Testing dataset LSTM (Difference)')
    axs[0, 1].set_xlabel(f'Time [{x_unit}]')
    axs[0, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[0, 1].legend()

    axs[1, 1].plot(val_results['Test Predictions'] - val_results['Actuals'], label='Difference', color='red')
    axs[1, 1].set_title('Testing dataset MLP ARX (Difference)')
    axs[1, 1].set_xlabel(f'Time [{x_unit}]')
    axs[1, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[1, 1].legend()

    axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
    axs[2, 1].set_title('Testing dataset LINEAR ARX (Difference)')
    axs[2, 1].set_xlabel(f'Time [{x_unit}]')
    axs[2, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[2, 1].legend()

    fig.suptitle(plot_name, fontsize=16)
    plt.show()


In [ ]:
#Porównanie wykresów wyników jednego modelu (train, val, test) + jednostki

def display_train_val_test(train_results, val_results, test_results, plot_name, y_unit='°C', x_unit='10s'):
    # Create a figure and subplots for each dataset from cross validation
    fig, axs = plt.subplots(
        3, 2,               
        figsize=(20, 16),    
        gridspec_kw=dict(hspace=0.5, wspace=0.5)
    )

    # Plot data on subplots
    axs[0, 0].plot(train_results['Train Predictions'], label='Model')
    axs[0, 0].plot(train_results['Actuals'], label='Real')
    axs[0, 0].set_title('Training dataset')
    axs[0, 0].set_xlabel(f'Time [{x_unit}]')
    axs[0, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[0, 0].legend()

    axs[1, 0].plot(val_results['Val Predictions'], label='Model')
    axs[1, 0].plot(val_results['Actuals'], label='Real')
    axs[1, 0].set_title('Validation dataset')
    axs[1, 0].set_xlabel(f'Time [{x_unit}]')
    axs[1, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[1, 0].legend()

    axs[2, 0].plot(test_results['Test Predictions'], label='Model')
    axs[2, 0].plot(test_results['Actuals'], label='Real')
    axs[2, 0].set_title('Testing dataset')
    axs[2, 0].set_xlabel(f'Time [{x_unit}]')
    axs[2, 0].set_ylabel(f'Temperature [{y_unit}]')
    axs[2, 0].legend()

    # Plot difference between model and real data
    axs[0, 1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
    axs[0, 1].set_title('Training dataset (Difference)')
    axs[0, 1].set_xlabel(f'Time [{x_unit}]')
    axs[0, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[0, 1].legend()

    axs[1, 1].plot(val_results['Val Predictions'] - val_results['Actuals'], label='Difference', color='red')
    axs[1, 1].set_title('Validation dataset (Difference)')
    axs[1, 1].set_xlabel(f'Time [{x_unit}]')
    axs[1, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[1, 1].legend()

    axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
    axs[2, 1].set_title('Testing dataset (Difference)')
    axs[2, 1].set_xlabel(f'Time [{x_unit}]')
    axs[2, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[2, 1].legend()

    fig.suptitle(plot_name, fontsize=16)
    plt.show()


In [ ]:
#Funkcja odpowiedzialna za wyświetlanie wyników z testing dataset
def display_test_results(test_results, plot_name):
    # Create a figure and subplots for the test dataset
    fig, axs = plt.subplots(
        2, 1,               # 2 rows, 1 column
        figsize=(20, 10),    # Width = 20, height = 10
        gridspec_kw=dict(hspace=0.5)   # Vertical spacing between subplots
    )

    # Plot predictions vs actuals
    axs[0].plot(test_results['Test Predictions'], label='Model Predictions')
    axs[0].plot(test_results['Actuals'], label='Actual Values')
    axs[0].set_title('Testing Dataset Predictions vs Actuals')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Value')
    axs[0].legend()

    # Plot the difference between predictions and actuals
    axs[1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Prediction Error', color='red')
    axs[1].set_title('Testing Dataset Prediction Error')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Value')
    axs[1].legend()

    # Set the main title and show the plot
    fig.suptitle(plot_name, fontsize=20)
    plt.show()


In [ ]:
def calculate_metrics(y_true, y_pred):
    me = (y_true - y_pred).mean()
    mse = ((y_true - y_pred) ** 2).mean()
    mae = (y_true - y_pred).abs().mean()
    rmse = np.sqrt(mse)
    return me, mse, mae, rmse

# DATA PREPARATION AND METHODS MLP ARX AND LINEAR ARX

In [ ]:
#imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#Resampling and adding shifted output values as inputs
#Model temperatury za schładzaczem 3
df_TS32P_MLP = df_TS32P_resampled
df_TS32P_MLP['y_k_1'] = df_TS32P_MLP['TS32P'].shift(1)  # y[k-1]
df_TS32P_MLP['y_k_2'] = df_TS32P_MLP['TS32P'].shift(2)  # y[k-2]
df_TS32P_MLP = df_TS32P_MLP.dropna().reset_index(drop=True)

#Model temperatury za przegrzewaczem 3
df_TP32P_MLP = df_TP32P_resampled
df_TP32P_MLP['y_k_1'] = df_TP32P_MLP['TP32P'].shift(1)  # y[k-1]
df_TP32P_MLP['y_k_2'] = df_TP32P_MLP['TP32P'].shift(2)  # y[k-2]
df_TP32P_MLP = df_TP32P_MLP.dropna().reset_index(drop=True)

#Model temperatury za schładzaczem 4
df_TS42P_MLP = df_TS42P_resampled
df_TS42P_MLP['y_k_1'] = df_TS42P_MLP['TS42P'].shift(1)  # y[k-1]
df_TS42P_MLP['y_k_2'] = df_TS42P_MLP['TS42P'].shift(2)  # y[k-2]
df_TS42P_MLP = df_TS42P_MLP.dropna().reset_index(drop=True)

#Model temperatury za przegrzewaczem 4
df_TP42P_MLP = df_TP42P_resampled
df_TP42P_MLP['y_k_1'] = df_TP42P_MLP['TP42P'].shift(1)  # y[k-1]
df_TP42P_MLP['y_k_2'] = df_TP42P_MLP['TP42P'].shift(2)  # y[k-2]
df_TP42P_MLP = df_TP42P_MLP.dropna().reset_index(drop=True)


In [ ]:
#Funkcja odpowiedzialna za tworzenie modelu MLP
def create_model_MLP(var_num):
    model = Sequential()
    model.add(Dense(64, activation = 'relu', input_dim = var_num))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
#Funkcja odpowiedzialna za tworzenie modelu MLP with one layer and linear activation function
def create_model_MLP_onelayer(var_num):
    model = Sequential()
    model.add(Dense(1, activation = 'linear', input_dim = var_num))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości MLP ARX-> wersja ze średnią ważoną
def predict_MLP(model, input, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    input = np.array(input) #We create a temporary variable for input
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict(input[i].reshape(1,input.shape[1]), verbose=0) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0][0]
        
        #Switching y[k-1] and y[k-2] values for those predicted by the model
        if i<num_steps-1:
           input[i+1][input.shape[1]-2] = ((9 * predictions[i] + labels[i]) / 10) #y[k-1] one before last column
           if i!=0:
               input[i+1][input.shape[1]-1] = ((9 * predictions[i-1] + labels[i-1]) / 10)#y[k-2] last column
           else:
               pass
        
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions


In [ ]:
# #Funkcja odpowiedzialna za przewidywanie kolejnych wartości MLP ARX-> wersja bez średniej ważonej
# def predict_MLP(model, input, num_steps):
#     predictions = np.zeros(num_steps) #empty numpy array for storing predictions
#     input = np.array(input) #We create a temporary variable for input
    
#     for i in range(num_steps):
        
#         #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
#         next_value = model.predict(input[i].reshape(1,input.shape[1]), verbose=0) 
        
#         #Adding predicted values to predictions array
#         predictions[i] = next_value[0][0]
        
#         #Switching y[k-1] and y[k-2] values for those predicted by the model
#         if i<num_steps-1:
#            input[i+1][input.shape[1]-2] = predictions[i]#y[k-1] one before last column
#            if i!=0:
#                input[i+1][input.shape[1]-1] = predictions[i-1]#y[k-2] last column
#            else:
#                pass
        
#         #Counting how many steps forward have already been predicted and printing it to the console
#         if i % 500 == 0:
#             #os.system('cls')
#             print(i, '/', num_steps)
#     print("Done")
#     return predictions


In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości LINEAR ARX-> wersja ze średnią ważoną
def predict_LINEAR(model, input, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    input = np.array(input) #We create a temporary variable for input
    
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict(input[i].reshape(1,input.shape[1])) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0]
        
        #Switching y[k-1] and y[k-2] values for those predicted by the model
    
        if i<num_steps-1: 
           input[i+1][input.shape[1]-2] = ((9 * predictions[i] + labels[i]) / 10) #y[k-1] one before last column
           if i!=0:
               input[i+1][input.shape[1]-1] = ((9 * predictions[i-1] + labels[i-1]) / 10)#y[k-2] last column
           else:
               pass
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions

In [ ]:
#columns for dataframe for each model as python lists
#Wyodrębnienie zbiorów danych niezbędnych do stworzenia każdego z modeli

#Model temperatury za schładzaczem 3
col_TS32P = ['TS32P', 'TS31P', 'GV3']

#Model temperatury za przegrzewaczem 3
col_TP32P= ['TP32P', 'TS32P', 'B', 'FP3P']

#Model temperatury za schładzaczem 4
col_TS42P = ['TS42P', 'GV4', 'FP3P','TP32P']

#Model temperatury za przegrzewaczem 4
col_TP42P = ['TP42P', 'TS42P', 'B', 'FP4P']

# MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3)

In [ ]:
#Comparison
display_comparison(get('test_results_TS32P_LSTM'), get('test_results_TS32P_MLP'), get('test_results_TS32P_LINEAR'), "MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3)")

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TS32P['TS32P'], WINDOW_SIZE)
var = df_TS32P[['TS31P','GV3']][WINDOW_SIZE-1:].to_numpy()

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape


In [ ]:
#Model creation
model1 = create_model(2) #2 additional variables not including sequences

cp = ModelCheckpoint('model_TS32P/model_TS32P.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model1.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions
model1 = load_model('model_TS32P/model_TS32P.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model1, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TS32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_TS32P, 'TS32P'),'train_results_TS32P')
# save(train_results_TS32P,'train_results_TS32P_normalized')

# val_predictions = predict(model1, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TS32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_TS32P, 'TS32P'),'val_results_TS32P')
# save(val_results_TS32P,'val_results_TS32P_normalized')

test_predictions = predict(model1, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TS32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS32P, 'TS32P'), 'test_results_TS32P')
save(test_results_TS32P,'test_results_TS32P_normalized')


In [ ]:
#Displaying results
display(get('train_results_TS32P'), get('val_results_TS32P'), get('test_results_TS32P'),'test', "MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3) LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TS32P_resampled['TS32P'], WINDOW_SIZE)
var = df_TS32P_resampled[['TS31P','GV3']][WINDOW_SIZE-1:].to_numpy()

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape


In [ ]:
#Model creation
model1 = create_model(2) #2 additional variables not including sequences

cp = ModelCheckpoint('model_TS32P/model_TS32P_LSTM_new.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model1.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=50, callbacks = [cp])

In [ ]:
#Model predictions
model1 = load_model('model_TS32P/model_TS32P_LSTM.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict(model1, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
train_results_TS32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS32P, 'TS32P'),'train_results_TS32P_LSTM')
save(train_results_TS32P,'train_results_TS32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(train_results_TS32P["Actuals"], train_results_TS32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict(model1, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_TS32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS32P, 'TS32P'),'val_results_TS32P_LSTM')
save(val_results_TS32P,'val_results_TS32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(val_results_TS32P["Actuals"], val_results_TS32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict(model1, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TS32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS32P, 'TS32P'), 'test_results_TS32P_LSTM')
save(test_results_TS32P,'test_results_TS32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
#display_test_results(get('test_results_TS32P_LSTM'), "MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3) LSTM")
display_train_val_test(get('train_results_TS32P_LSTM'), get('val_results_TS32P_LSTM'), get('test_results_TS32P_LSTM'), "MODEL 1 LSTM - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3)")

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
size = df_TS32P_MLP.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df_TS32P_MLP[col_TS32P[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df_TS32P_MLP[col_TS32P[0]][:train]
y_train = y_train.to_numpy()

X_val = df_TS32P_MLP[col_TS32P[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df_TS32P_MLP[col_TS32P[0]][train:val]
y_val = y_val.to_numpy()

X_test = df_TS32P_MLP[col_TS32P[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df_TS32P_MLP[col_TS32P[0]][val:]
y_test = y_test.to_numpy()

In [ ]:
#Model creation MLP
model1 = create_model_MLP(4) #2+2y

cp = ModelCheckpoint('model_TS32P/model_TS32P_MLP_new.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training MLP
model1.fit(X_train, y_train, validation_data=(X_val,  y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions MLP
model1 = load_model('model_TS32P/model_TS32P_MLP.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_MLP(model1, X_train, window_train, y_train)
train_results_TS32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS32P, 'TS32P'),'train_results_TS32P_MLP')
save(train_results_TS32P,'train_results_TS32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(train_results_TS32P["Actuals"], train_results_TS32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_MLP(model1, X_val, window_val, y_val)
val_results_TS32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS32P, 'TS32P'),'val_results_TS32P_MLP')
save(val_results_TS32P,'val_results_TS32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(val_results_TS32P["Actuals"], val_results_TS32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_MLP(model1, X_test, window_test, y_test)
test_results_TS32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS32P, 'TS32P'), 'test_results_TS32P_MLP')
save(test_results_TS32P,'test_results_TS32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
#me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#LINEAR
#Model creation linear
model1 = LinearRegression()
#Model training linear
model1.fit(X_train, y_train)
#model1 = load_model('model_TS32P/model_TS32P_LINEAR.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_LINEAR(model1, X_train, window_train, y_train)
train_results_TS32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS32P, 'TS32P'),'train_results_TS32P_LINEAR')
save(train_results_TS32P,'train_results_TS32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(train_results_TS32P["Actuals"], train_results_TS32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_LINEAR(model1, X_val, window_val, y_val)
val_results_TS32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS32P, 'TS32P'),'val_results_TS32P_LINEAR')
save(val_results_TS32P,'val_results_TS32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(val_results_TS32P["Actuals"], val_results_TS32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_LINEAR(model1, X_test, window_test, y_test)
test_results_TS32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS32P, 'TS32P'), 'test_results_TS32P_LINEAR')
save(test_results_TS32P,'test_results_TS32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TS32P["Actuals"], test_results_TS32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
display_test_results(get('test_results_TS32P_MLP'), "MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3) MLP ARX")

In [ ]:
display_train_val_test(get('train_results_TS32P_MLP'), get('val_results_TS32P_MLP'), get('test_results_TS32P_MLP'), "MODEL 1 MLP ARX - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3)")

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_TS32P_LINEAR'), "MODEL 1 - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3) LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_TS32P_LINEAR'), get('val_results_TS32P_LINEAR'), get('test_results_TS32P_LINEAR'), "MODEL 1 LINEAR ARX - Model temperatury za schładzaczem 3 - TS32P (TS31P, GV3)")

## Metrics

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 1 - Model temperatury za schładzaczem 3 - TS32P', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/TS32P_models_metrics_NEW.csv", index=False)

In [ ]:
def highlight_min_green(s):
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]

In [ ]:
metrics_df.style.apply(highlight_min_green, subset=[''])

In [ ]:
metrics_df = pd.read_csv("metrics/TS32P_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P)

In [ ]:
#Comparison
display_comparison(get('test_results_TP32P_LSTM'), get('test_results_TP32P_MLP'), get('test_results_TP32P_LINEAR'), "MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TP32P, B, FP3P)")

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TP32P['TP32P'], WINDOW_SIZE)
#df_TP32P['TS32P'] = df_TP32P['TS32P'].shift(-5).fillna(method = 'bfill')
var = df_TP32P[['TS32P', 'B', 'FP3P']][WINDOW_SIZE-1:].to_numpy()


X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model2 = create_model(3) #3 additional variables

cp = ModelCheckpoint('model_TP32P/model_TP32P.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model2 = load_model('model_TP32P/model_TP32P.keras')
model2.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions
model2 = load_model('model_TP32P/model_TP32P.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model2, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TP32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_TP32P,'TP32P'),'train_results_TP32P')
# save(train_results_TP32P, 'train_results_TP32P_normalized')

# val_predictions = predict(model2, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TP32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_TP32P, 'TP32P'),'val_results_TP32P')
# save(val_results_TP32P, 'val_results_TP32P_normalized')

test_predictions = predict(model2, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TP32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP32P, 'TP32P'), 'test_results_TP32P')
save(test_results_TP32P, 'test_results_TP32P_normalized')

In [ ]:
#Displaying results
display(get('train_results_TP32P'), get('val_results_TP32P'), get('test_results_TP32P_raw_data'),'test', "MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P) LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TP32P_resampled['TP32P'], WINDOW_SIZE)
#df_TP32P['TS32P'] = df_TP32P['TS32P'].shift(-5).fillna(method = 'bfill')
var = df_TP32P_resampled[['TS32P', 'B', 'FP3P']][WINDOW_SIZE-1:].to_numpy()

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model2 = create_model(3) #3 additional variables

cp = ModelCheckpoint('model_TP32P/model_TP32P_LSTM.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model2 = load_model('model_TP32P/model_TP32P_LSTM.keras')
model2.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=20, callbacks = [cp])

In [ ]:
#Model predictions
model2 = load_model('model_TP32P/model_TP32P_LSTM.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict(model2, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
train_results_TP32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP32P, 'TP32P'),'train_results_TP32P_LSTM')
save(train_results_TP32P,'train_results_TP32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(train_results_TP32P["Actuals"], train_results_TP32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict(model2, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_TP32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP32P, 'TP32P'),'val_results_TP32P_LSTM')
save(val_results_TP32P,'val_results_TP32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(val_results_TP32P["Actuals"], val_results_TP32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict(model2, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TP32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP32P, 'TP32P'), 'test_results_TP32P_LSTM')
save(test_results_TP32P,'test_results_TP32P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
display_test_results(get('test_results_TP32P_LSTM'), "MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P) LSTM")


In [ ]:
display_train_val_test(get('train_results_TP32P_LSTM'), get('val_results_TP32P_LSTM'), get('test_results_TP32P_LSTM'), "MODEL 2 LSTM - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P)")

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
size = df_TP32P_MLP.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df_TP32P_MLP[col_TP32P[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df_TP32P_MLP[col_TP32P[0]][:train]
y_train = y_train.to_numpy()

X_val = df_TP32P_MLP[col_TP32P[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df_TP32P_MLP[col_TP32P[0]][train:val]
y_val = y_val.to_numpy()

X_test = df_TP32P_MLP[col_TP32P[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df_TP32P_MLP[col_TP32P[0]][val:]
y_test = y_test.to_numpy()

In [ ]:
#Model creation
model2 = create_model_MLP(5) #3+2y

cp = ModelCheckpoint('model_TP32P/model_TP32P_MLP.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model2 = load_model('model_TP32P/model_TP32P_MLP.keras')
model2.fit(X_train, y_train, validation_data=(X_val,  y_val), epochs=20, callbacks = [cp])

In [ ]:
#Model predictions MLP
model2 = load_model('model_TP32P/model_TP32P_MLP.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_MLP(model2, X_train, window_train, y_train)
train_results_TP32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP32P, 'TP32P'),'train_results_TP32P_MLP')
save(train_results_TP32P,'train_results_TP32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(train_results_TP32P["Actuals"], train_results_TP32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_MLP(model2, X_val, window_val, y_val)
val_results_TP32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP32P, 'TP32P'),'val_results_TP32P_MLP')
save(val_results_TP32P,'val_results_TP32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(val_results_TP32P["Actuals"], val_results_TP32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_MLP(model2, X_test, window_test, y_test)
test_results_TP32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP32P, 'TP32P'), 'test_results_TP32P_MLP')
save(test_results_TP32P,'test_results_TP32P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#LINEAR
#Model creation linear
model2 = LinearRegression()
#Model training linear
model2.fit(X_train, y_train)
#model2 = load_model('model_TP32P/model_TP32P_LINEAR.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_LINEAR(model2, X_train, window_train, y_train)
train_results_TP32P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP32P, 'TP32P'),'train_results_TP32P_LINEAR')
save(train_results_TP32P,'train_results_TP32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(train_results_TP32P["Actuals"], train_results_TP32P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_LINEAR(model2, X_val, window_val, y_val)
val_results_TP32P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP32P, 'TP32P'),'val_results_TP32P_LINEAR')
save(val_results_TP32P,'val_results_TP32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(val_results_TP32P["Actuals"], val_results_TP32P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_LINEAR(model2, X_test, window_test, y_test)
test_results_TP32P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP32P, 'TP32P'), 'test_results_TP32P_LINEAR')
save(test_results_TP32P,'test_results_TP32P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP32P["Actuals"], test_results_TP32P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
display_test_results(get('test_results_TP32P_MLP'), "MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P) MLP ARX")

In [ ]:
display_train_val_test(get('train_results_TP32P_MLP'), get('val_results_TP32P_MLP'), get('test_results_TP32P_MLP'), "MODEL 2 MLP ARX - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P)")

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_TP32P_LINEAR'), "MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P) LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_TP32P_LINEAR'), get('val_results_TP32P_LINEAR'), get('test_results_TP32P_LINEAR'), "MODEL 2 LINEAR ARX - Model temperatury za przegrzewaczem 3 - TP32P (TS32P, B, FP3P)")

## Metrics

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 2 - Model temperatury za przegrzewaczem 3 - TP32P', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/TP32P_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/TP32P_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P)

In [ ]:
#Comparison
display_comparison(get('test_results_TS42P_LSTM_test'), get('test_results_TS42P_MLP'), get('test_results_TS42P_LINEAR'), "MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P)")

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TS42P['TS42P'], WINDOW_SIZE)
var = df_TS42P[['GV4', 'FP3P', 'TP32P']][WINDOW_SIZE-1:].to_numpy()  #, 'TS41P'='TS32P'

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model3 = create_model(3) #3 additional variables ('TS41P'='TS32P')

cp = ModelCheckpoint('model_TS42P/model_TS42P.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model3.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions
model3 = load_model('model_TS42P/model_TS42P.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model3, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TS42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(train_results_TS42P,'train_results_TS42P_normalized')
# save(denorm(train_results_TS42P,'TS42P'),'train_results_TS42P')

# val_predictions = predict(model3, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TS42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(val_results_TS42P,'val_results_TS42P_normalized')
# save(denorm(val_results_TS42P,'TS42P'),'val_results_TS42P')

test_predictions = predict(model3, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TS42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(test_results_TS42P, 'test_results_TS42P_normalized')
save(denorm(test_results_TS42P,'TS42P'), 'test_results_TS42P')

In [ ]:
#Displaying results
display(get('train_results_TS42P'), get('val_results_TS42P'), get('test_results_TS42P'),'test', "MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P)")

## LSTM resampled data

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TS42P_resampled['TS42P'], WINDOW_SIZE)
var = df_TS42P_resampled[['GV4', 'FP3P', 'TP32P']][WINDOW_SIZE-1:].to_numpy()  #, 'TS41P'='TS32P'

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model3 = create_model(3) #3 additional variables ('TS41P'='TS32P')

cp = ModelCheckpoint('model_TS42P/model_TS42P_LSTM.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model3 = load_model('model_TS42P/model_TS42P_LSTM.keras')
model3.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions
model3 = load_model('model_TS42P/model_TS42P_LSTM.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict(model3, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
train_results_TS42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS42P, 'TS42P'),'train_results_TS42P_LSTM')
save(train_results_TS42P,'train_results_TS42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(train_results_TS42P["Actuals"], train_results_TS42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict(model3, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_TS42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS42P, 'TS42P'),'val_results_TS42P_LSTM')
save(val_results_TS42P,'val_results_TS42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(val_results_TS42P["Actuals"], val_results_TS42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict(model3, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TS42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS42P, 'TS42P'), 'test_results_TS42P_LSTM')
save(test_results_TS42P,'test_results_TS42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
display_test_results( get('test_results_TS42P_LSTM'), "MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P) LSTM")

In [ ]:
display_train_val_test(get('train_results_TS42P_LSTM'), get('val_results_TS42P_LSTM'), get('test_results_TS42P_LSTM'), "MODEL 3 LSTM - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P))")

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
size = df_TS42P_MLP.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df_TS42P_MLP[col_TS42P[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df_TS42P_MLP[col_TS42P[0]][:train]
y_train = y_train.to_numpy()

X_val = df_TS42P_MLP[col_TS42P[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df_TS42P_MLP[col_TS42P[0]][train:val]
y_val = y_val.to_numpy()

X_test = df_TS42P_MLP[col_TS42P[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df_TS42P_MLP[col_TS42P[0]][val:]
y_test = y_test.to_numpy()

In [ ]:
#Model creation
model3 = create_model_MLP(5) #3+2y

cp = ModelCheckpoint('model_TS42P/model_TS42P_MLP.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model3.fit(X_train, y_train, validation_data=(X_val,  y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions MLP
model3 = load_model('model_TS42P/model_TS42P_MLP.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_MLP(model3, X_train, window_train, y_train)
train_results_TS42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS42P, 'TS42P'),'train_results_TS42P_MLP')
save(train_results_TS42P,'train_results_TS42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(train_results_TS42P["Actuals"], train_results_TS42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_MLP(model3, X_val, window_val, y_val)
val_results_TS42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS42P, 'TS42P'),'val_results_TS42P_MLP')
save(val_results_TS42P,'val_results_TS42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(val_results_TS42P["Actuals"], val_results_TS42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_MLP(model3, X_test, window_test, y_test)
test_results_TS42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS42P, 'TS42P'), 'test_results_TS42P_MLP')
save(test_results_TS42P,'test_results_TS42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#LINEAR
#Model creation linear
model3 = LinearRegression()
#Model training linear
model3.fit(X_train, y_train)
#model3 = load_model('model_TS42P/model_TS42P_LINEAR.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_LINEAR(model3, X_train, window_train, y_train)
train_results_TS42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TS42P, 'TS42P'),'train_results_TS42P_LINEAR')
save(train_results_TS42P,'train_results_TS42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(train_results_TS42P["Actuals"], train_results_TS42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_LINEAR(model3, X_val, window_val, y_val)
val_results_TS42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TS42P, 'TS42P'),'val_results_TS42P_LINEAR')
save(val_results_TS42P,'val_results_TS42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(val_results_TS42P["Actuals"], val_results_TS42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_LINEAR(model3, X_test, window_test, y_test)
test_results_TS42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TS42P, 'TS42P'), 'test_results_TS42P_LINEAR')
save(test_results_TS42P,'test_results_TS42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TS42P["Actuals"], test_results_TS42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results MLP
display_test_results(get('test_results_TS42P_MLP'), "MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P) MLP ARX")

In [ ]:
display_train_val_test(get('train_results_TS42P_MLP'), get('val_results_TS42P_MLP'), get('test_results_TS42P_MLP'), "MODEL 3 MLP ARX - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P))")

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_TS42P_LINEAR'), "MODEL 3 - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P) LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_TS42P_LINEAR'), get('val_results_TS42P_LINEAR'), get('test_results_TS42P_LINEAR'), "MODEL 3 LINEAR ARX - Model temperatury za schładzaczem 4 - TS42P (TS41P, GV4, FP3P))")

## Metrics

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 3 - Model temperatury za schładzaczem 4 - TS42P', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/TS42P_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/TS42P_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)

In [ ]:
#Comparison
display_comparison(get('test_results_TP42P_LSTM'), get('test_results_TP42P_MLP'), get('test_results_TP42P_LINEAR'), "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)")

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TP42P['TP42P'], WINDOW_SIZE)
var = df_TP42P[['TS42P', 'B', 'FP4P']][WINDOW_SIZE-1:].to_numpy()

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model4 = create_model(3) #3 Additional variables

cp = ModelCheckpoint('model_TP42P/model_TP42P_raw_data.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model4.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions
model4 = load_model('model_TP42P/model_TP42P_raw_data.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model4, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(train_results_TP42P,'train_results_TP42P_normalized')
# save(denorm(train_results_TP42P,'TP42P'),'train_results_TP42P')

# val_predictions = predict(model4, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(val_results_TP42P,'val_results_TP42P_normalized')
# save(denorm(val_results_TP42P,'TP42P'),'val_results_TP42P')

test_predictions = predict(model4, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(test_results_TP42P, 'test_results_TP42P_normalized_raw_data')
#save(denorm(test_results_TP42P,'TP42P'), 'test_results_TP42P')

In [ ]:
#Displaying results
display(get('train_results_TP42P'), get('val_results_TP42P'), get('test_results_TP42P_raw_data'),'test', "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)")

## LSTM resampled data

In [ ]:
#Data preparation
WINDOW_SIZE = 10
X,y = data_to_X_y(df_TP42P_resampled['TP42P'], WINDOW_SIZE)
var = df_TP42P_resampled[['TS42P', 'B', 'FP4P']][WINDOW_SIZE-1:].to_numpy()

X_train, var_train, y_train, X_val, var_val, y_val, X_test, var_test, y_test = data_to_cross_val(X, var, y)

#X_train.shape, var_train.shape, y_train.shape, X_val.shape, var_val.shape, y_val.shape, X_test.shape, var_test.shape, y_test.shape

In [ ]:
#Model creation
model4 = create_model(3) #3 Additional variables

cp = ModelCheckpoint('model_TP42P/model_TP42P_LSTM.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model4.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=50, callbacks = [cp])

In [ ]:
#Model predictions
model4 = load_model('model_TP42P/model_TP42P_LSTM.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict(model4, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP42P, 'TP42P'),'train_results_TP42P_LSTM')
save(train_results_TP42P,'train_results_TP42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(train_results_TP42P["Actuals"], train_results_TP42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict(model4, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP42P, 'TP42P'),'val_results_TP42P_LSTM')
save(val_results_TP42P,'val_results_TP42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(val_results_TP42P["Actuals"], val_results_TP42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict(model4, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP42P, 'TP42P'), 'test_results_TP42P_LSTM')
save(test_results_TP42P,'test_results_TP42P_normalized_LSTM')

me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results
display_test_results(get('test_results_TP42P_LSTM'), "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P) LSTM")

In [ ]:
display_train_val_test(get('train_results_TP42P_LSTM'), get('val_results_TP42P_LSTM'), get('test_results_TP42P_LSTM'), "MODEL 4 LSTM - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)")

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
size = df_TP42P_MLP.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df_TP42P_MLP[col_TP42P[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df_TP42P_MLP[col_TP42P[0]][:train]
y_train = y_train.to_numpy()

X_val = df_TP42P_MLP[col_TP42P[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df_TP42P_MLP[col_TP42P[0]][train:val]
y_val = y_val.to_numpy()

X_test = df_TP42P_MLP[col_TP42P[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df_TP42P_MLP[col_TP42P[0]][val:]
y_test = y_test.to_numpy()

In [ ]:
#Model creation
model4 = create_model_MLP(5) #3+2y

cp = ModelCheckpoint('model_TP42P/model_TP42P_MLP.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model4.fit(X_train, y_train, validation_data=(X_val,  y_val), epochs=10, callbacks = [cp])

In [ ]:
#Model predictions MLP
model4 = load_model('model_TP42P/model_TP42P_MLP.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_MLP(model4, X_train, window_train, y_train)
train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP42P, 'TP42P'),'train_results_TP42P_MLP')
save(train_results_TP42P,'train_results_TP42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(train_results_TP42P["Actuals"], train_results_TP42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_MLP(model4, X_val, window_val, y_val)
val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP42P, 'TP42P'),'val_results_TP42P_MLP')
save(val_results_TP42P,'val_results_TP42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(val_results_TP42P["Actuals"], val_results_TP42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_MLP(model4, X_test, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP42P, 'TP42P'), 'test_results_TP42P_MLP')
save(test_results_TP42P,'test_results_TP42P_normalized_MLP')

me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#LINEAR
#Model creation linear
model4 = LinearRegression()
#Model training linear
model4.fit(X_train, y_train)
#model4 = load_model('model_TP42P/model_TP42P_LINEAR.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

train_predictions = predict_LINEAR(model4, X_train, window_train, y_train)
train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_TP42P, 'TP42P'),'train_results_TP42P_LINEAR')
save(train_results_TP42P,'train_results_TP42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(train_results_TP42P["Actuals"], train_results_TP42P["Train Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

val_predictions = predict_LINEAR(model4, X_val, window_val, y_val)
val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_TP42P, 'TP42P'),'val_results_TP42P_LINEAR')
save(val_results_TP42P,'val_results_TP42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(val_results_TP42P["Actuals"], val_results_TP42P["Val Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

test_predictions = predict_LINEAR(model4, X_test, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP42P, 'TP42P'), 'test_results_TP42P_LINEAR')
save(test_results_TP42P,'test_results_TP42P_normalized_LINEAR')

me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})


In [ ]:
# me, mse, mae, rmse = calculate_metrics(test_results_TP42P["Actuals"], test_results_TP42P["Test Predictions"])
# metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
#Displaying results MLP
display_test_results(get('test_results_TP42P_MLP'), "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P) MLP ARX")

In [ ]:
display_train_val_test(get('train_results_TP42P_MLP'), get('val_results_TP42P_MLP'), get('test_results_TP42P_MLP'), "MODEL 4 MLP ARX - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)")

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_TP42P_LINEAR'), "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P) LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_TP42P_LINEAR'), get('val_results_TP42P_LINEAR'), get('test_results_TP42P_LINEAR'), "MODEL 4 LINEAR ARX - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P)")

## Metrics

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/TP42P_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/TP42P_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# Code check

In [ ]:
def difference_equation_with_inputs(a1, a2, b1, b2, b3, x0, x1, input1, input2, input3, noise, n):
    """
    Calculates values for a difference equation with three inputs:
    x[i] = a1 * x[i-1] + a2 * x[i-2] + b1 * input1[i] + b2* input2[i] + b3* input3[i]) + noise
    
    Parameters:
    a1, a2: autoregression coefficients
    b1, b2, b3 : coefficients for input variables
    c: constant term
    x0, x1: initial conditions
    input1, input2, input3: input arrays
    n: number of iterations
    """
    x = np.zeros(n)
    x[0], x[1] = x0, x1
    
    for i in range(2, n):
        x[i] = a1 * x[i-1] + a2 * x[i-2] + b1 * input1[i] + b2* input2[i] + b3* input3[i] + noise[i]
    
    return x

In [ ]:
# Generate example input data
n = 5000
input1 = np.sin(np.linspace(0, 5*np.pi, n))  # Sinusoidal signal
input2 = -abs(np.cos(np.linspace(0, 10*np.pi, n ))/1.5)+0.25 #Cosinusoidal signal
input3 = np.cumsum(np.random.randn(n))/10  # Scaled random walk
noise = np.random.randn(n)/25  # Gaussian noise
# Equation parameters
a1, a2 = 0.5, -0.2  # autoregression coefficients
b1, b2, b3 = -0.5, 0.7, 0.25 # coefficients for input variables
#c = 0.2  # constant term
x0, x1 = 0, 0.5  # initial conditions

# Calculate values
x = difference_equation_with_inputs(a1, a2, b1, b2, b3, x0, x1, input1, input2, input3, noise, n)

# Visualize results
plt.figure(figsize=(12, 10))

plt.subplot(5, 1, 1)
plt.plot(range(n), x, 'b-')
plt.title('Difference Equation Output')
plt.ylabel('x[n]')

plt.subplot(5, 1, 2)
plt.plot(range(n), input1, 'g-')
plt.title('Input 1 (Sinusoidal)')
plt.ylabel('input1[n]')

plt.subplot(5, 1, 3)
plt.plot(range(n), input2, 'r-')
plt.title('Input 2 (Cosinusoidal)')
plt.ylabel('input2[n]')

plt.subplot(5, 1, 4)
plt.plot(range(n), input3, 'm-')
plt.title('Input 3 (Scaled Random Walk)')
plt.xlabel('n')
plt.ylabel('input3[n]')

plt.subplot(5, 1, 5)
plt.plot(range(n), noise, 'y-')
plt.title('Noise (Gaussian noise)')
plt.xlabel('n')
plt.ylabel('noise')

plt.tight_layout()
plt.show()

# Print first 10 values
print("First 10 values:")
for i in range(10):
    print(f"x[{i}] = {x[i]:.4f}")


In [ ]:
x

In [ ]:
df = pd.DataFrame({
  'x': x,
  'input1': input1,
  'input2': input2,
  'input3': input3
})
col = list(df.columns)

In [ ]:
df

In [ ]:
df['y_k_1'] = df['x'].shift(1)  # y[k-1]
df['y_k_2'] = df['x'].shift(2)  # y[k-2]
df = df.dropna().reset_index(drop=True)

In [ ]:
df

In [ ]:
ranges = {column:(min(df[column]),max(df[column])) for column in df.columns}

In [ ]:
ranges

In [ ]:
df_raw = df
df_raw.to_csv("check/df_raw_1.csv")

In [ ]:
df

In [ ]:
noise_raw = noise

In [ ]:
noise_raw

In [ ]:
noise

In [ ]:

for column in df:
  if(ranges[column] != (0, 0)): #jeśli jest podany zakres
    df[[column]] = (df[[column]]-ranges[column][0])/(ranges[column][1]-ranges[column][0])
noise = (noise_raw-noise_raw.min())/(noise_raw.max()-noise_raw.min())
df.head()

In [ ]:
df.to_csv("check/df_norm_1.csv")

In [ ]:
#Data preparation (normalized data)
size = df.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df[col[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df[col[0]][:train]
y_train = y_train.to_numpy()

X_val = df[col[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df[col[0]][train:val]
y_val = y_val.to_numpy()


X_test = df[col[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df[col[0]][val:]
y_test = y_test.to_numpy()


noise_train = noise[2:train+2]
noise_val = noise[train+2:val+2]
noise_test = noise[val+2:]




In [ ]:
pd.DataFrame(X_train, columns=['input1', 'input2', 'input3', 'y_k_1', 'y_k_2']).to_csv("check/X_train_norm.csv")
pd.DataFrame(X_val, columns=['input1', 'input2', 'input3', 'y_k_1', 'y_k_2']).to_csv("check/X_val_norm.csv")
pd.DataFrame(X_test, columns=['input1', 'input2', 'input3', 'y_k_1', 'y_k_2']).to_csv("check/X_test_norm.csv")
pd.DataFrame(y_train, columns=['y']).to_csv("check/y_train_norm.csv")
pd.DataFrame(y_val, columns=['y']).to_csv("check/y_val_norm.csv")
pd.DataFrame(y_test, columns=['y']).to_csv("check/y_test_norm.csv")
pd.DataFrame(noise_train, columns=['Gaussian noise']).to_csv("check/noise_train_norm.csv")
pd.DataFrame(noise_val, columns=['Gaussian noise']).to_csv("check/noise_val_norm.csv")
pd.DataFrame(noise_test, columns=['Gaussian noise']).to_csv("check/noise_test_norm.csv")

In [ ]:
#Data check (raw data)
size = df_raw.shape[0]
train = int(size*0.8)
val = int(size*0.9)

X_train = df_raw[col[1:] + ['y_k_1','y_k_2']][:train]
X_train = X_train.to_numpy()
y_train = df_raw[col[0]][:train]
y_train = y_train.to_numpy()

X_val = df_raw[col[1:] + ['y_k_1','y_k_2']][train:val]
X_val = X_val.to_numpy()
y_val = df_raw[col[0]][train:val]
y_val = y_val.to_numpy()

X_test = df_raw[col[1:] + ['y_k_1','y_k_2']][val:]
X_test = X_test.to_numpy()
y_test = df_raw[col[0]][val:]
y_test = y_test.to_numpy()

noise_train = noise_raw[2:train+2]
noise_val = noise_raw[train+2:val+2]
noise_test = noise_raw[val+2:]


In [ ]:
df_raw = pd.read_csv("check/df_raw_1.csv")

In [ ]:
df_raw

In [ ]:

a1, a2 = 0.5, -0.2  # autoregression coefficients
b1, b2, b3 = -0.5, 0.7, 0.25 # coefficients for input variables

i = 90
input1 = 0
input2 = 1
input3 = 2
y_k_1 = 3
y_k_2 = 4

x = a1 * X_test[i][y_k_1] + a2 * X_test[i][y_k_2] + b1 * X_test[i][input1] + b2* X_test[i][input2] + b3* X_test[i][input3] + noise_test[i]
print(x)
print(y_test[i])

In [ ]:
y_test

In [ ]:
print(X_test[i][input1])
print(X_test[i][input2])
print(X_test[i][input3])
print(X_test[i][y_k_1])
print(X_test[i][y_k_2])

In [ ]:
a1, a2 = 0.5, -0.2  # autoregression coefficients
b1, b2, b3 = -0.5, 0.7, 0.25 # coefficients for input variables


x = a1 * 0.5 + a2 * 0 + b1 * 0.00628440082918321 + b2* -0.4166140084082909 + b3* 0.07316128932018406 + noise[2]
print(x)

In [ ]:
#Model creation
model4 = create_model_MLP_onelayer(5) #3+2y

cp = ModelCheckpoint('check/MLP_onelayer_check.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
model4.fit(X_train, y_train, validation_data=(X_val,  y_val), epochs=30, callbacks = [cp])

In [ ]:
#Model predictions
model4 = load_model('check/MLP_onelayer_check.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model4, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_TP42P, 'TP42P'),'train_results_TP42P_MLP')
# save(train_results_TP42P,'train_results_TP42P_normalized_MLP')

# val_predictions = predict(model4, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_TP42P, 'TP42P'),'val_results_TP42P_MLP')
# save(val_results_TP42P,'val_results_TP42P_normalized_MLP')

test_predictions = predict_MLP(model4, X_test, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP42P, 'x'), 'test_results_MLP_onelayer_check')
save(test_results_TP42P,'test_results_MLP_onelayer_check_norm')


In [ ]:
#LINEAR
#Model creation linear
model_linear = LinearRegression()
#Model training linear
model_linear.fit(X_train, y_train)
#model_linear = load_model('model_TP42P/model_TP42P_LINEAR.keras')

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
val = 0
window_val = X_val.shape[0]
test = 0
window_test = X_test.shape[0]

# train_predictions = predict(model_linear, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_TP42P = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_TP42P, 'TP42P'),'train_results_TP42P_LINEAR')
# save(train_results_TP42P,'train_results_TP42P_normalized_LINEAR')

# val_predictions = predict(model_linear, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_TP42P = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_TP42P, 'TP42P'),'val_results_TP42P_LINEAR')
# save(val_results_TP42P,'val_results_TP42P_normalized_LINEAR')

test_predictions = predict_LINEAR(model_linear, X_test, window_test, y_test)
test_results_TP42P = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_TP42P, 'x'), 'test_results_LinearRegression_check')
save(test_results_TP42P,'test_results_LinearRegression_check_norm')


In [ ]:
model4.weights[0][0].numpy()[0]

In [ ]:
list(model_linear.coef_)

In [ ]:
weights = pd.DataFrame({
  'model': ['diff_equation', 'MLP_ARX', 'LINEAR_ARX'],
  'input1_weight': [b1, model4.weights[0][0].numpy()[0], list(model_linear.coef_)[0]],
  'input2_weight': [b2, model4.weights[0][1].numpy()[0], list(model_linear.coef_)[1]],
  'input3_weight': [b3, model4.weights[0][2].numpy()[0], list(model_linear.coef_)[2]],
  'y_k_1_weight': [a1, model4.weights[0][3].numpy()[0], list(model_linear.coef_)[3]],
  'y_k_2_weight': [a2, model4.weights[0][4].numpy()[0], list(model_linear.coef_)[4]]
  
})
weights.head()

In [ ]:
weights.to_csv('check/weights.csv')

In [ ]:
weights = pd.DataFrame({
  'model': ['diff_equation', 'MLP_ARX', 'LINEAR_ARX'],
  'input1_weight': [0.3, 0.2550321, 0.08607239],
  'input2_weight': [0.3, 0.13437146, 0.33218012],
  'input3_weight': [0.3, 0.5927429,0.53543508],
  'y_k_1_weight': [0.5, -0.23844196, 0.5],
  'y_k_2_weight': [-0.2, 0.16206977, -0.2]
  
})
weights.head()

In [ ]:
#Displaying results MLP
display(get('train_results_TP42P'), get('val_results_TP42P'), get('test_results_MLP_onelayer_check'),'test', "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P) MLP ARX")

In [ ]:
#Displaying results LINEAR
display(get('train_results_TP42P'), get('val_results_TP42P'), get('test_results_LinearRegression_check'),'test', "MODEL 4 - Model temperatury za przegrzewaczem 4 - TP42P (TS42P, B, FP4P) LINEAR ARX")